In [ ]:
#Importing libraries
import numpy as np
from itertools import count
from scipy.optimize import linprog #used to run the
import time

#Generating the constraint matrix for the number scenarios and the percentages
def generate_constraint_matrix(num_scenarios,percentage):
  #Initializing lists and counts
  constraint_matrices = []
  constraints = []
  Slacks = []
  variables_count = 3 + 6*num_scenarios
  slacks_count = 1 + 4*num_scenarios
  c = [0] * (variables_count + slacks_count)
  b = [0] * (1 + 4*num_scenarios)

#The code uses nested loops to generate the constraint matrices (constraints and Slacks), coefficient vector (c), and right-hand side vector (b).
  for i in range(num_scenarios):
    # Generate constraints for each scenario
    for j in range(5):  # Number of constraints per scenario
      row = [0] * variables_count
      if i==0 and j == 0:
        row[0] = 1  # Coefficients for X1 in the first constraint
        row[1] = 1  # Coefficients for X2 in the first constraint
        row[2] = 1  # Coefficients for X3 in the first constraint
      elif i!=0 and j == 0:
        continue  # Skip the first constraint for subsequent matrices
      elif j == 1:
        row[0] = 2.5 * (1+percentage*((num_scenarios-1)/2)-percentage*i)   # Coefficients for X1 in the second constraint
        row[3+6*i] = 1  # Coefficients for X4 in the second constraint
        row[5+6*i] = -1  # Coefficients for X6 in the second constraint
      elif j == 2:
        row[1] = 3 * (1+percentage*((num_scenarios-1)/2)-percentage*i)  # Coefficients for X2 in the third constraint
        row[4+6*i] = 1  # Coefficients for X5 in the third constraint
        row[6+6*i] = -1  # Coefficients for X7 in the third constraint
      elif j == 3:
        row[2] = -20 * (1+percentage*((num_scenarios-1)/2)-percentage*i)  # Coefficients for X3 in the fourth constraint
        row[7+6*i] = 1  # Coefficients for X8 in the fourth constraint
        row[8+6*i] = 1  # Coefficients for X9 in the fourth constraint
      elif j == 4:
        row[7+6*i] = 1  # Coefficients for X8 in the fifth constraint

      constraints.append(row)

  for i in range(num_scenarios):
    # Generate constraints for each scenario
    for j in range(5):  # Number of constraints per scenario
      row = [0] * slacks_count
      if i==0 and j == 0:
        row[0] = 1  # Coefficients for s1 in the first constraint
      elif i!=0 and j == 0:
        continue  # Skip the first constraint for subsequent matrices
      elif j == 1:
        row[1+4*i] = -1  # Coefficients for s2 in the second constraint
      elif j == 2:
        row[2+4*i] = -1  # Coefficients for s3 in the third constraint
      elif j == 3:
        row[3+4*i] = 1  # Coefficients for s4 in the fourth constraint
      elif j == 4:
        row[4+4*i] = 1  # Coefficients for s5 in the fifth constraint

      Slacks.append(row)

#This section of code constructs the final constraint matrix by concatenating the constraints and slack variables horizontally.
  constraint_matrices = np.hstack((constraints,Slacks))
  #Defining the original unchanged variables
  c[0] = 150
  c[1] = 230
  c[2] = 260

  #The code uses nested loops to generate the coefficient vector (c).
  for i in range(num_scenarios):
    c[3 + (i*6)] = 238 * (1/num_scenarios)
    c[4 + (i*6)] = 210 * (1/num_scenarios)
    c[5 + (i*6)] = -170 * (1/num_scenarios)
    c[6 + (i*6)] = -150 * (1/num_scenarios)
    c[7 + (i*6)] = -36 * (1/num_scenarios)
    c[8 + (i*6)] = -10 * (1/num_scenarios)

  #The code uses nested loops to generate the right-hand side vector (b).
  b[0] = 500
  for i in range(num_scenarios):
    b[1 + (i*4)] = 200
    b[2 + (i*4)] = 240
    b[3 + (i*4)] = 0
    b[4 + (i*4)] = 6000
#This code converts the b list to a NumPy array and reshapes it to a column vector.
  b = np.array(b).reshape(-1,1)
#This code returns the generated constraint matrices (constraint_matrices), objective function coefficients (c), and right-hand side vector (b).
  return constraint_matrices, c, b

In [ ]:
#This code generates the coefficients for the phase 1 simplex method

#
def generate_phase_one_lp_coefficients_from_constraints(A,b, c):
    #The code below determines the number of constraints (num_constraints) and the number of decision variables (num_variables) based on the shape of the A matrix.
    num_constraints, num_variables = A.shape
    #This code initializes a vector x of ones with a shape of (num_variables, 1).
    x = np.ones((num_variables,1))
    #Calculating the "artificial_matrix" by taking the identity matrix multiplied by the difference between the right-hand side vector b and the result of the matrix-vector multiplication A @ x.
    artificial_matrix = np.eye(num_constraints) * (b - (A@x))
    #Concatenates the original constraint matrix A and the artificial matrix horizontally to create a new matrix A_with_artificial.
    A_with_artificial = np.concatenate((A, artificial_matrix), axis=1)
    #It initializes a new coefficient vector c_new with zeros and a length of num_variables + num_constraints.
    c_new = np.zeros(num_variables + num_constraints)
    #It assigns the first num_variables elements of c_new with the values from the original coefficient vector c.
    c_new[:num_variables] = c
    #It assigns the remaining elements (corresponding to the artificial variables) in c_new with a large constant value (1,000,000 in this case).
    c_new[num_variables:] = 1000000  # Coefficients for artificial variables

    #Organizing the generated coefficients
    coefficients = {
        'A': A_with_artificial,
        'b': b.reshape(-1,1),
        'c': c_new.reshape(-1,1)}

    return coefficients

In [ ]:
#Initializing the number of scenarios to percentage
num_scenarios = 1
percentage = 0.001
e = 0.0001
A_pre,c_pre,b_pre = generate_constraint_matrix(num_scenarios,percentage)

In [ ]:
beginning = time.time()
phase_one_coefficients = generate_phase_one_lp_coefficients_from_constraints(A_pre,b_pre,c_pre)

A = phase_one_coefficients['A']
b = phase_one_coefficients['b']
c = phase_one_coefficients['c']

zero_X = np.zeros((A_pre.shape[1],1))
x = np.ones((A.shape[1],1))

for i_count in count():
  X_k = np.diag(np.squeeze(x))
  y = np.linalg.solve(X_k, x)

  W_left = A @ (X_k @ X_k) @ A.T
  W_right = A @ (X_k @ X_k) @ c
  W_k = np.linalg.solve(W_left, W_right)
  r_k = c - A.T @ W_k

  gap = y.T @ X_k @ r_k

  if abs(gap) <= e:
    Optimal_X = x
    Optimal_value = c.T @ x
    break

  direction = -X_k @ r_k
  direction = direction.reshape(-1, 1)
  alpha = np.array([])

  for d in direction:
    if d < 0:
      a = 0.6
      a = -(a / d)
      alpha = np.append(alpha,a)

  a_min = np.min(alpha)


  x = x + a_min * (X_k @ direction)

total_time = time.time() - beginning

In [ ]:
Optimal_value

array([[-112978.94217645]])

In [ ]:
total_time

9294.201314687729

In [ ]:
beginning2 = time.time()
result = linprog(c, A_eq = A, b_eq = b, method = 'interior-point')
total_time2 = time.time() - beginning2

<ipython-input-117-27cb5182994a>:2: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_eq = A, b_eq = b, method = 'interior-point')


In [ ]:
result.fun

In [ ]:
total_time2